# Домашняя работа
### Тема: Коллаборативная фильтрация

**Пакет SURPRISE**

*   Используйте данные MovieLens 1M
*   Можно использовать любые модели из пакета
*   получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :  
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [1]:
!pip install surprise

In [2]:
from surprise import Dataset, accuracy, Reader, SVD, KNNBaseline
from surprise.model_selection import train_test_split, KFold
from statistics import mean
import pandas as pd

In [3]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv(r'ratings.csv')

In [4]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [5]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [6]:
dataset.head()

,uid,iid,rating
0,4.0,Toy Story (1995),4.0
1,10.0,Toy Story (1995),5.0
2,14.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),4.0
4,22.0,Toy Story (1995),4.0


In [7]:
dataset.rating.min(), dataset.rating.max()

(0.5, 5.0)

In [8]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [9]:
trainset, testset = train_test_split(data, test_size=.60)

In [10]:
kf = KFold(n_splits=5)

**Пробуем SVD**

In [11]:
algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8881
RMSE: 0.8810
RMSE: 0.8799
RMSE: 0.8839
RMSE: 0.8762


С SVD получили не самый лучший RMSE, так что пробуем другой вариант.

**Пробуем KNN**

In [13]:
algo = KNNBaseline(k=30, sim_options={'name': 'pearson_baseline', 'user_based': False})

accuracy_n = []
n = 0
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset, verbose=False)
    n += 1
    accuracy_n.append(accuracy.rmse(predictions, verbose=False))
    print('RMSE: ', accuracy.rmse(predictions, verbose=False))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE:  0.8662551627268441
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE:  0.8714965830177839
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE:  0.8698431593831857
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE:  0.8633030356405679
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE:  0.8588901425415719


KNN показал себя лучше и мы получили необходимые значения.